# Coleta, Tratamento e Unificação das bases de dados relativas as Usinas Sucroalcooleiras

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [1]:
%run nb00_Setup_Usinas{}

In [2]:
%run nb_conn_db_mercado_potencial

# Leitura dos Dados 

In [3]:
# usinas   = pd.read_parquet(abfss_path_enriched+'usinas/usi01_gov_usinas.parquet',
#                             storage_options = {'linked_service' : linked_service_enriched})


usinas = ConsultaView('VW_GOV_USINAS')

## Inclusão de Infos Geográficas

In [4]:
url ='https://servicodados.ibge.gov.br/api/v1/localidades/estados'
estados = requests.get(url, verify=False).json()

In [5]:
# colentando cod nomes UFs
#uf = pd.json_normalize(Estados().json_ibge)
uf = pd.json_normalize(estados)

# agregando infos geograficas as usinas
usinas = usinas.merge(uf[['id','sigla','regiao.id','regiao.sigla']], how='left', left_on='UF', right_on='sigla')
usinas.rename(columns={'id':'ID_UF','regiao.id':'ID_REGIAO','regiao.sigla':'REGIAO'}, inplace=True)
usinas.drop('sigla', axis=1, inplace=True)


# Exploração

In [6]:
col = ['CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL', 'RECEITA_OP_CC', 'N_EMPREG_CC','TANCAGEM_DIESEL']
usinas[col] = usinas[col].apply(pd.to_numeric)
usinas[col].isnull().sum()/477

In [7]:
# Compute the correlation matrix
corr = usinas[col].corr(method='pearson')

# Generate a mask for the upper triangle
# mask = np.triu(np.ones_like(corr, dtype=bool))

# plt.figure(figsize = (12,8))
# sns.heatmap(corr, mask=mask, annot=True, cmap='Greens')
# plt.show()

In [8]:
#msno.matrix(usinas[col],figsize=(12,8), fontsize=12)

In [9]:
#msno.bar(usinas[col], figsize=(12,8), fontsize=12)

# Imputações

## Processo 1 - Imputação das Tancagens ANP

Processo sequencial de imputação por colunas do dataframe. Para que fique evidente quais os vizinhos doadores doaram dados o último processo será o de Tancagem de Diesel.

In [10]:
usinas1 = usinas.copy()

In [11]:
var         = ['CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL']
usinas_impt = usinas[['CNPJ','UF','REGIAO','CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL']]
usinas_impt.sort_values('CNPJ', ignore_index=True, inplace=True)

# IMPUTANDO ATÉ TANCAGEM ETANOL
usinas_impt = agrega_imputados(dados=usinas_impt,
                               variaveis=var,
                               chave='CNPJ',
                               alvo='TANCAGEM_ETANOL',
                               nivel=['UF', 'REGIAO'],
                               viz=5,
                               ind=False)

In [12]:
usinas1.set_index('CNPJ', inplace=True)
usinas1.sort_index(inplace=True)
usinas1.update(usinas_impt.set_index('CNPJ'), overwrite=False)
usinas1.reset_index(drop=False, inplace=True)

## Processo 2 - Imputação das Tancagem DIESEL - VIBRA

In [13]:
usinas2 = usinas1.copy()

In [14]:
var         = ['CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL']
usinas_impt = usinas2[['CNPJ','UF','REGIAO','CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL','TANCAGEM_DIESEL']]
usinas_impt.sort_values('CNPJ', ignore_index=True, inplace=True)

# IMPUTANDO ATÉ TANCAGEM DIESEL
usinas_impt = agrega_imputados(dados=usinas_impt,
                               variaveis=var,
                               chave='CNPJ',
                               alvo='TANCAGEM_DIESEL',
                               nivel=['UF', 'REGIAO'],
                               viz=5,
                               ind=True)

#usinas_impt = agrega_imputados(usinas_impt, var, chave='CNPJ', alvo='', nivel=['UF', 'REGIAO'], viz=5)

In [15]:
usinas2.set_index('CNPJ', inplace=True)
usinas2.sort_index(inplace=True)
usinas2.update(usinas_impt.set_index('CNPJ'),overwrite=False)
usinas2.reset_index(drop=False, inplace=True)

## Imputando dados em Receita e Empregados

In [16]:
usinas3 = usinas1.copy()

Verificar a proporcionalidade de usinas dentro de consumidor final

In [17]:
# ASSUMINDO QUE AS RECEITAS OPERACIONAIS FALTANTES ESTEJAM NA MENOR CLASSE

usinas3['RECEITA_OP_CC'] = usinas3['RECEITA_OP_CC'].fillna(0)
usinas3['RECEITA_OP_CC'] = usinas3['RECEITA_OP_CC'].where(usinas3['RECEITA_OP_CC']!=-1,0)
#usinas3['RECEITA_OP_CC'].value_counts()

# = usinas3['RECEITA_OP_C'].fillna('0-50')

In [18]:
var         = ['CAP_HIDRATADO','CAP_ANIDRO', 'TANCAGEM_ETANOL', 'RECEITA_OP_CC', 'N_EMPREG_CC',]
usinas_impt = usinas3[['CNPJ','UF','REGIAO','CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL', 'RECEITA_OP_CC', 'N_EMPREG_CC']]
usinas_impt.sort_values('CNPJ', ignore_index=True, inplace=True)

# IMPUTANDO ATÉ TANCAGEM ETANOL
usinas_impt = agrega_imputados(usinas_impt, var, chave='CNPJ', alvo='N_EMPREG_CC', nivel=['UF', 'REGIAO'], viz=5, ind=False)

In [19]:
usinas3.set_index('CNPJ', inplace=True)
usinas3.sort_index(inplace=True)
usinas3.update(np.round(usinas_impt).set_index('CNPJ'),overwrite=False)
usinas3.reset_index(drop=False, inplace=True)

In [20]:
# testar se é necessário !!!
usinas_impt['RECEITA_OP_CC'] = usinas_impt['RECEITA_OP_CC'].where(usinas_impt['RECEITA_OP_CC']!=-1,np.nan)
usinas_impt['N_EMPREG_CC'] = usinas_impt['N_EMPREG_CC'].where(usinas_impt['N_EMPREG_CC']!=-1,np.nan)

### Parte 3 - Imputando dados de Tanque Diesel

In [21]:
usinas4 = usinas3.copy()

In [22]:
col         = ['CNPJ','UF','REGIAO','CAP_HIDRATADO','CAP_ANIDRO','TANCAGEM_ETANOL', 'RECEITA_OP_CC', 'N_EMPREG_CC','TANCAGEM_DIESEL']
var         = ['CAP_HIDRATADO','CAP_ANIDRO', 'TANCAGEM_ETANOL', 'RECEITA_OP_CC', 'N_EMPREG_CC','TANCAGEM_DIESEL']
usinas_impt = usinas4[col]
usinas_impt.sort_values('CNPJ', ignore_index=True, inplace=True)

usinas_impt = agrega_imputados(usinas_impt, var, chave='CNPJ', alvo='TANCAGEM_DIESEL', nivel=['UF', 'REGIAO'], viz=5, ind=False)

In [23]:
usinas4.set_index('CNPJ', inplace=True)
usinas4.sort_index(inplace=True)
usinas4.update(usinas_impt.set_index('CNPJ'),overwrite=False)
usinas4.reset_index(drop=False, inplace=True)

In [24]:
usinas2[col]

In [25]:
usinas4[col]

In [26]:
x = usinas2['TANCAGEM_DIESEL']-usinas4['TANCAGEM_DIESEL']
pd.cut(x, bins=15).value_counts()

Verificou-se que as informação ede receita bruta e de número de empregados não traz informações capazez de mudar significativament as imputações dos volumes de Diesel realizadas pelos volumes de Etanol

# Salvando 

In [27]:
# usinas2.drop(['RECEITA_OP_C', 'RECEITA_OP_CC','N_EMPREG_C', 'N_EMPREG_CC'], axis=1,inplace=True)
usinas2 = usinas2.loc [:,~usinas2.columns.str.contains  ('Recei|Empre', case=False)]

In [28]:
usinas2.to_parquet(abfss_path_enriched + 'usinas/usi02_tancagens_impt_usinas.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})